In [2]:
%pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import dotenv
import os

dotenv_path = '.env'
dotenv.load_dotenv(dotenv_path)

NAVER_CLIENT_ID = os.getenv('NAVER_API_CLIENT_ID')
NAVER_CLIENT_SECRET = os.getenv('NAVER_API_CLIENT_SECRET')

In [ ]:
import urllib.request
url = "https://openapi.naver.com/v1/datalab/search"
# body = "{\"startDate\":\"2017-01-01\",\"endDate\":\"2017-04-30\",\"timeUnit\":\"month\",\"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},{\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}],\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"}";
body = "{\
         \"startDate\":\"2022-10-01\",\
         \"endDate\":\"2022-10-04\",\
         \"timeUnit\":\"date\",\
         \"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},\
                             {\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}\
                            ],\
         \"device\":\"pc\",\
         \"ages\":[\"1\",\"2\"],\
         \"gender\":\"f\"\
         }"



request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",NAVER_CLIENT_ID)
request.add_header("X-Naver-Client-Secret",NAVER_CLIENT_SECRET)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"startDate":"2022-10-01","endDate":"2022-10-04","timeUnit":"date","results":[{"title":"한글","keywords":["한글","korean"],"data":[{"period":"2022-10-01","ratio":61.4035},{"period":"2022-10-02","ratio":49.1228},{"period":"2022-10-03","ratio":84.21052},{"period":"2022-10-04","ratio":100}]},{"title":"영어","keywords":["영어","english"],"data":[{"period":"2022-10-01","ratio":24.5614},{"period":"2022-10-02","ratio":12.2807},{"period":"2022-10-03","ratio":17.54385},{"period":"2022-10-04","ratio":15.78947}]}]}


In [14]:
import requests
import urllib.request
# import urllib.parse

def format_url_text(text):
    return urllib.parse.quote(text)

def naver_search_api(query):
    format_text = format_url_text(query)

    url = f"https://openapi.naver.com/v1/datalab/search/"
    headers = {
        'X-Naver-Client-Id': NAVER_CLIENT_ID,
        'X-Naver-Client-Secret': NAVER_CLIENT_SECRET,
        'Content-Type': 'application/json'
    }
    params = {
        'query': format_text,
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [27]:
import json
import requests
import pandas as pd

class Naver:
    def __init__(self, client_id, client_secret):
        self.headers = {
            "X-Naver-Client-Id": client_id,
            "X-Naver-Client-Secret": client_secret,
            "Content-Type": "application/json",
        }

    # API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/search/search.md
    def datalab_search(self, startDate, endDate, timeUnit, keywordGroups, **kwargs):
            # 네이버 통합 검색어 트렌드 조회
            # startDate: string        # 조회 시작일
            # endDate : string         # 조회 종료일
            # timeUnit : string        # 조회 기간
            # keywordGroups : list     # 키워드 그룹
            # kwargs : dict            # 추가 옵션
            # Returns type: DataFrame  # 키워드 그룹별 트렌드 데이터   
            
            url = "https://openapi.naver.com/v1/datalab/search"
            data = {
                "startDate": startDate,
                "endDate": endDate,
                "timeUnit": timeUnit,
                "keywordGroups": keywordGroups,
            }
            data.update(kwargs)
            data = json.dumps(data, ensure_ascii=False).encode("utf-8")
            res = requests.post(url, data=data, headers=self.headers)
            if res.status_code == 200:
                data = res.json()['results']
                num = len(data)
                df = pd.DataFrame()
                for i in range(num):
                    sub = pd.DataFrame(data[i]['data'])
                    sub['title'] = data[i]['title']
                    sub = sub[['title', 'period', 'ratio']]
                    df = pd.concat([df, sub], axis=0, ignore_index=True)
                pivot = pd.pivot(df, index='period', columns='title',
                                values='ratio').reset_index()
                pivot.columns.name = None
                pivot = pivot.rename(columns={"period": "날짜"})
                return pivot
            else:
                return res

In [24]:
naver_trends = Naver(NAVER_CLIENT_ID, NAVER_CLIENT_SECRET)

In [29]:
naver_trends.datalab_search("2024-10-01", "2024-10-04", "date", 
                            [{"groupName": "감기", "keywords": ["감기", "기침", "독감"]}, 
                             {"groupName": "온열질환", "keywords": ["온열질환", "열병", "열사병"]}])

,날짜,감기,온열질환
0,2024-10-01,66.96948,30.02562
1,2024-10-02,94.17656,8.57209
2,2024-10-03,70.16072,5.45073
3,2024-10-04,100.00000,6.68530
